In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [2]:
mol_str = """
0 1
#O
#H 1 1.1
#H 1 1.1 2 104
#n
#n 1 1.0
#li
#h 1 1.0
#o
#h 1 1.0
#h 1 1.2 2 140.0
be
h 1 1.0
h 1 1.2 2 104.0
symmetry c1
"""

options_dict = {'basis': '6-31g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [3]:
# photon energy
omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
#omega_val = 0

print(omega_val)

# lambda vector
#lambda_vector = np.array([0., 0., 0.0])
lambda_vector = np.array([0.5, 0.5, 0.5])
#number of active orbitals
n_act_orb=5
#number of active electrons
n_act_el=4

# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(1, mol_str, options_dict, lambda_vector,omega_val, n_act_el,n_act_orb,ignore_coupling=False, cas=False)
nmo=H2_PF.nmo
ndocc=H2_PF.ndocc
numDets=H2_PF.numDets
excitation_index=H2_PF.excitation_index
print(nmo)
print(ndocc)
print(numDets)
print(excitation_index)
# uncomment to print
#print(H2_PF.H_PF)


0.07349864496170931

Start SCF iterations:

Canonical RHF One-electron energy = -28.4911549036043077
CQED-RHF One-electron energy      = -26.2654749621615053
Nuclear repulsion energy          = 4.1851092680938411
Dipole energy                     = 1.2045254616481860
SCF Iteration   1: Energy = -13.9239831619187395   dE = -1.39240E+01   dRMS = 7.75367E-02
SCF Iteration   2: Energy = -14.3972422939082154   dE = -4.73259E-01   dRMS = 3.19053E-02
SCF Iteration   3: Energy = -14.4391376999961150   dE = -4.18954E-02   dRMS = 1.07338E-02
SCF Iteration   4: Energy = -14.4453479856950349   dE = -6.21029E-03   dRMS = 5.22332E-03
SCF Iteration   5: Energy = -14.4464425316910603   dE = -1.09455E-03   dRMS = 1.84375E-03
SCF Iteration   6: Energy = -14.4466617695638444   dE = -2.19238E-04   dRMS = 9.75378E-04
SCF Iteration   7: Energy = -14.4467095810974282   dE = -4.78115E-05   dRMS = 3.75265E-04
SCF Iteration   8: Energy = -14.4467207756929934   dE = -1.11946E-05   dRMS = 2.06321E-04
SCF Iteratio

In [4]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

#print the ground-state and first excited state
#print("Psi4 |G> is ",fci_e)
print("|G> is ",e_fci[0])
print("|LP> is ",e_fci[1])
nsingdet=numDets-1
#print(wavefunctions)
c00 = wavefunctions[0,3]
c10 = wavefunctions[1:nsingdet+1,3]
c01 = wavefunctions[nsingdet+1,3]
c11 = wavefunctions[(nsingdet+2):,3]
print(wavefunctions[:,3])
print(c10)
print(c11)
print(c11.shape)
print('aaaa')
#print(c11)
nvirt=nmo-ndocc
c10_large =np.zeros((4*ndocc*nvirt))
c11_large =np.zeros((4*ndocc*nvirt))
for n in range(nsingdet):
    ia = excitation_index[n]
    a = ia%(2*nvirt)
    i = (ia-a)//(2*nvirt)
    print (i,a+2*ndocc,excitation_index[n])
    c10_large[excitation_index[n]]=c10[n]
    c11_large[excitation_index[n]]=c11[n]
        
    
print(c10_large)
Dij = np.zeros((2*ndocc,2*ndocc))
Dab = np.zeros((2*nvirt,2*nvirt))
for i in range(2*ndocc):
    for j in range(2*ndocc):
        Dij[i,j] = (c00 * c00 + c01 * c01) * (i==j)
        Dij[i,j] += (np.dot(c10.T,c10)+np.dot(c11.T,c11)) * (i==j)
        dum = 0.0
        for a in range(2*nvirt):
            ia = i * 2*nvirt + a
            ja = j * 2*nvirt + a
            dum += c10_large[ia] * c10_large[ja]
            dum += c11_large[ia] * c11_large[ja]
        Dij[i,j] -=dum    

#print(Dij)
print(np.trace(Dij))
for a in range(2*nvirt):
    for b in range(2*nvirt):
        dum = 0.0
        for i in range(2*ndocc):
            ia = i * 2*nvirt + a
            ib = i * 2*nvirt + b
            dum += c10_large[ia] * c10_large[ib]
            dum += c11_large[ia] * c11_large[ib]
        Dab[a,b] += dum    
print(np.trace(Dab))             
print(np.trace(Dij)+np.trace(Dab))            

|G> is  -14.462780350520507
|LP> is  -14.390744794751031
[ 1.48730702e-03 -5.82860401e-01 -4.60996443e-02 -3.44083391e-02
 -7.90763961e-02 -7.54763448e-03  2.82531229e-02 -1.20326016e-02
  4.18077012e-03 -5.27898601e-03 -6.59571695e-03  2.88880920e-02
 -5.48201192e-04  5.57859068e-03  8.76125421e-03 -3.48125196e-03
 -5.06197380e-03  2.50178200e-03  2.48413156e-03  2.87840459e-04
  1.00206885e-03 -3.10389769e-04  2.09762417e-04 -4.24839019e-04
 -2.98315814e-04 -1.73314496e-05  1.73700842e-04 -6.39165055e-05
 -2.01896302e-05 -5.07719863e-05  1.63754600e-05 -5.82860401e-01
 -4.60996443e-02 -3.44083391e-02 -7.90763961e-02 -7.54763448e-03
  2.82531229e-02 -1.20326016e-02  4.18077012e-03 -5.27898601e-03
 -6.59571695e-03  2.88880920e-02 -5.48201192e-04  5.57859068e-03
  8.76125421e-03 -3.48125196e-03 -5.06197380e-03  2.50178200e-03
  2.48413156e-03  2.87840459e-04  1.00206885e-03 -3.10389769e-04
  2.09762417e-04 -4.24839019e-04 -2.98315814e-04 -1.73314496e-05
  1.73700842e-04 -6.39165055e-05 

In [ ]:

# First compute SCF energy using Psi4
scf_e, wfn = psi4.energy('SCF', return_wfn=True)

# now compute cqed-rhf to get transformation vectors with cavity
cqed_rhf_dict = cqed_rhf(lambda_vector, mol_str, options_dict)
rhf_reference_energy = cqed_rhf_dict["RHF ENERGY"]
cqed_reference_energy = cqed_rhf_dict["CQED-RHF ENERGY"]
C = cqed_rhf_dict["CQED-RHF C"]



# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)
#psi4.set_options({'restricted_docc': [0],'active': [5]})
#fci_energy = psi4.energy('fci',ref_wfn=wfn)

ndocc = 5
n_act_el = 4
n_act_orb = 4

n_in_orb=ndocc-n_act_el//2
n_ac_el_half=n_act_el//2
print(n_ac_el_half)
inactive_list=list(x for x in range(n_in_orb))
print(inactive_list)
print('Generating all determinants in active space')
t = time.time()
singlesDets = []
for alpha in combinations(range(n_act_orb), n_ac_el_half):
    alphalist=list(alpha)
    alphalist=[x+n_in_orb for x in alphalist]
    alphalist[0:0]=inactive_list
    alpha=tuple(alphalist)
    #alpha_ex_level = compute_excitation_level(alpha, ndocc)
    for beta in combinations(range(n_act_orb), n_ac_el_half):
        betalist=list(beta)
        betalist=[x+n_in_orb for x in betalist]
        betalist[0:0]=inactive_list
        beta=tuple(betalist)
        #beta_ex_level = compute_excitation_level(beta, ndocc)
        #if alpha_ex_level + beta_ex_level == 1:
        print(F' adding alpha: {alpha} and beta: {beta}\n') 
        singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))